In [1]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# File management #

In [1]:
#data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")
data=pd.read_excel("C:/Users/jortuzar/Documents/Git/nanonis/Order.xls")

NameError: name 'pd' is not defined

In [21]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
#path=data.iloc[:,10]
path=data.iloc[:,11]
Zapproach_fil=data.iloc[:,1]

In [22]:
#dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
dat=[ 0,1,2,3,4,5,6,7, 8, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 24,32]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]
for j in dat:
    #T.append([float(i) for i in temp[j].split(',')])
    #Tfiles.append([path[j]+i for i in temp_files[j].split(',')])
    size.append(size_dat[j])
    height.append(height_dat[j])
    B.append([int(i) for i in Bfields[j].split(',')])
    Bfiles.append([path[j]+i for i in Bfield_files[j].split(',')])
    Jfiles.append([path[j]+i for i in josephson[j].split(',')])


for j in range(len(B)):
    list1, list2 = zip(*sorted(zip(B[j], Bfiles[j])))
    B[j]=np.array(list1)
    Bfiles[j]=np.array(list2)
size=np.array(size)
height=np.array(height)

Zapproach=[]
Zapproach.append([path[32]+i for i in Zapproach_fil[32].split(',')])

In [59]:
i=10
spectra.load(Jfiles[i][-1])
x=spectra.bias
y=spectra.conductance
plt.plot(x,y/y[0],label=str(size[i]))
spectra.load(Jfiles[i][0])
x=spectra.bias
y=spectra.conductance
plt.plot(x,y/y[0])
plt.legend()

In [123]:
Bfield_files[0]

'S230726_052.dat,S230726_055.dat,S230726_065.dat,S230726_066.dat,S230726_077.dat,S230726_078.dat,S230726_089.dat,S230726_090.dat,S230726_101.dat,S230726_102.dat,S230726_115.dat'

In [60]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1.0
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1.0
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1.0

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(delta_t,t,eta)*(1-fermi(T,t)),bcs_i(Delta,x,eta,n)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(bcs(delta_s,t,eta)*(1-fermi(T,t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)

In [117]:
x=np.linspace(-5,5,100)
y=currentD2(x,10000,1,0.01,0.4,-0.8,1.3,1.3,0.3,0.11,0.05)
plt.plot(x,y/y[0])
i=10
spectra.load(Jfiles[i][0])
x=spectra.bias
y=spectra.conductance
plt.plot(1000*x,y/y[0],label=str(size[i]))

plt.legend()

In [119]:
h=0
for i in np.linspace(1,10,10):
    x=np.linspace(-10,10,100)
    y=currentD2(x,i,1,0.01,0.4,0.8,1.3,1.3,0.3,0.11,0.05)
    plt.plot(x,y/y[0]+h)
    h+=1

In [122]:
q0=np.linspace(-1,1,10)
p0,pn,p_n=all_pD(0,1000,1,0.01,0.4,q0,1.3,1.3,0.3,0.11,0.05,2)
plt.scatter(q0,p0)

C:\Users\jortuzar\AppData\Local\Temp\ipykernel_19064\4068234084.py:44: RuntimeWarning: divide by zero encountered in divide
  mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_19064\4068234084.py:44: RuntimeWarning: invalid value encountered in divide
  mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
C:\Users\jortuzar\AppData\Local\Temp\ipykernel_19064\4068234084.py:45: RuntimeWarning: divide by zero encountered in divide
  en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,

TypeError: object of type 'int' has no len()